# Enabled User Right in AD to Control User Objects
Detects scenario where if a user is assigned the SeEnableDelegationPrivilege right in Active Directory it would allow control of other AD user objects.

## Rule Content
```
- title: Enabled User Right in AD to Control User Objects
  id: 311b6ce2-7890-4383-a8c2-663a9f6b43cd
  description: Detects scenario where if a user is assigned the SeEnableDelegationPrivilege
    right in Active Directory it would allow control of other AD user objects.
  tags:
  - attack.privilege_escalation
  - attack.t1078
  references:
  - https://www.harmj0y.net/blog/activedirectory/the-most-dangerous-user-right-you-probably-have-never-heard-of/
  author: '@neu5ron'
  logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : Policy Change > Audit Authorization
      Policy Change, Group Policy : Computer Configuration\Windows Settings\Security
      Settings\Advanced Audit Policy Configuration\Audit Policies\Policy Change\Audit
      Authorization Policy Change'
    category: null
  detection:
    selection:
      EventID: 4704
    keywords:
      Message:
      - '*SeEnableDelegationPrivilege*'
    condition: all of them
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4704" AND Message.keyword:(*SeEnableDelegationPrivilege*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()